In [ ]:
!pip3 install moviepy

In [ ]:
import mmcv
import numpy as np
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.utils import auto_select_device
device = auto_select_device()

config_file =  "work_dirs/mask_rcnn_balloon/mask_rcnn_balloon.py"
checkpoint_file = "work_dirs/mask_rcnn_balloon/latest.pth"

model = init_detector(config_file, checkpoint_file,device)

In [ ]:
def colour_mask_grayscale(image, result, thr = 0.65):
    
    img_copy = image.copy()
    
    bbox = result[0][0]
    seg = result[1][0]
    masks = np.zeros([img_copy.shape[:2][0], img_copy.shape[:2][1]])
    
    n_obj = bbox.shape[0]
    
    for i in range(n_obj):
        if bbox[i, -1] < thr:
            # print(bbox[i, -1])
            continue
    
        masks += seg[i]
        #print(masks.shape)
    
    mask_img = img_copy*masks.astype(bool)[:, :, None]
    img_gray = np.dot(img_copy[...,:3], [0.299, 0.587, 0.114])
    img_gray[masks==1] = 0
    
    return (mask_img+img_gray[:, :, None]).astype(np.uint8)

def process(image):
    
    #img = mmcv.imread(image)
    
    result = inference_detector(model, image)
    
    return colour_mask_grayscale(image, result)

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
clip = VideoFileClip("test_video.mp4")

In [ ]:
print(clip.duration)
print(clip.fps)

In [ ]:
masked_clip = clip.fl_image(process) #NOTE: this function expects color images!!
%time masked_clip.write_videofile("output.mp4", fps=25, audio=False)